In [29]:
import re
from sys import argv, exit


def get_tagpairs(sentence):
    sentence = sentence.strip()
    wordlist = re.split(r'(?<=>)(.+?)(?=<)', sentence)
    raw_taglist = wordlist[::4]

    taglist = [x[1:-1] for x in raw_taglist]

    phrases = map(str.strip, wordlist[1::4])
    tagged = map(lambda x: tag_items(x[0], x[1]), zip(taglist, phrases))

    return tagged


def tag_items(tag, phrase):
    HEADER = 'B-{}'.format(tag)
    TRAILER = 'L-{}'.format(tag)
    MIDDLE = 'I-{}'.format(tag)
    SINGLE = 'U-{}'.format(tag)

    words = [x.lower() for x in phrase.split()]

    if tag == 'O':
        return [[x, 'O'] for x in words]
    
    if len(words) == 0:
        return [[x, 'O'] for x in words]
    
    if len(words) == 1:
        return [[words[0], SINGLE], ]

    if len(words) == 2:
        return [[words[0], HEADER], [words[1], TRAILER]]

    intermediaries = [[x, MIDDLE] for x in words[1:-1]]

    result = []
    result.extend([[words[0], HEADER], ])
    result.extend(intermediaries)
    result.extend([[words[-1], TRAILER], ])

    return result



In [37]:
file = "../grammars/train.txt"
outfile = "samples/train.bilou"

In [38]:
with open(file) as f:
        data = f.read()

In [39]:
ldata = [x for x in data.split('\n') if x]

In [40]:
final = []
for x in ldata:
    if x[0] != "<":
        continue
    raw_processed = sum(get_tagpairs(x), [])
    processed = [' '.join(x) for x in raw_processed]
    final.append(processed)

In [41]:
print(final[0])

['i O', 'want O', 'to O', 'go O', 'to O', '11 U-Addr-Building-Number', 'airport B-Addr-Street', 'road L-Addr-Street', '#3-700 U-Addr-UnitNo', 'singapore U-Addr-city']


In [42]:
DELIMITER = '\n\n-DOCSTART- O\n\n'
final = ['\n'.join(x) for x in final]

final_output = DELIMITER[2:] + DELIMITER.join(final) + '\n'

with open(outfile, 'w') as f:
    f.write(final_output)


In [43]:
!pwd

/home/groot/InverseTextNormalization/scripts
